In [54]:
import selenium
import pandas as pd
from selenium import webdriver
import warnings
warnings.filterwarnings('ignore')
from selenium.webdriver.common.by import By
import time
from selenium.common.exceptions import NoSuchElementException

In [28]:
# connecting webdriver
driver=webdriver.Chrome(r"chromedriver.exe")

# Given url
url='https://www.imdb.com/search/title/?genres=action&sort=user_rating,desc&title_type=feature&num_votes=25000,&pf_rd_m=A2FGELUUNOQJNL&pf_rd_p=f11158cc-b50b-4c4d-b0a2-40b32863395b&pf_rd_r=XZ8X52H1R40B7KG5SNZ9&pf_rd_s=right-6&pf_rd_t=15506&pf_rd_i=top&ref_=chttp_gnr_1'

# Opening Url
driver.get(url)

# maximize window
driver.maximize_window()
time.sleep(2)


In [18]:
# Scrapping Snos
sno=[]
m_name=[]
year=[]
certificate=[]
runtime=[]
genre=[]
rating=[]
meta=[]
director=[]
star=[]
vote=[]
gross=[]
urls=[]
popularity=[]
for page in range(0,36):
    
    # scraping Sno, Movie name, Year
    smy= driver.find_elements(By.XPATH, '//h3[@class="lister-item-header"]')
    for i in smy:
        sno.append(i.text.split(".")[0])
        m_name.append(i.text.split(".")[1].split("(")[0].strip(" "))
        year.append(i.text.split('(')[-1].strip(")"))
     
    # scraping Certifiaction, Duration, Genre
    crg=driver.find_elements(By.XPATH, '//p[@class="text-muted "]')    
    for i in crg:
        try:
            certificate.append(i.text.split("|")[0])
        except:
            certificate.append('NA')
        try:    
            runtime.append(i.text.split("|")[1])
        except:
            runtime.append('NA')
        try:
            genre.append(i.text.split('|')[2])
        except:
            genre.append('NA')
            
     # scraping movie rating       
    ratings=driver.find_elements(By.XPATH, '//*[@id="main"]/div/div[3]/div//strong')    
    for i in ratings:    
        rating.append(i.text)
        
     # scraping metascore         
    metascore=driver.find_elements(By.XPATH, '//div[@class="ratings-bar"]')    
    for i in metascore:
        try:
            meta.append(i.text.split("\n")[1].split(" ")[0]) 
        except:
            meta.append('NA')  
            
    # scraping movie directors, stars
    ds=driver.find_elements(By.XPATH, '//*[@id="main"]/div/div[3]/div//p[3]')    
    for i in ds:
        try:
            director.append(i.text.split("|")[0].split(":")[1])
        except:
            director.append('NA')
        try:
            star.append(i.text.split("|")[1].split(":")[1])
        except:
            star.append('NA')
            
    # scraping votes and Gross earnings
    votes=driver.find_elements(By.XPATH, '//*[@id="main"]/div/div[3]/div//p[4]/span[2]')    
    for i in votes:
        vote.append(i.text)
        
    grossv=driver.find_elements(By.XPATH, '//*[@id="main"]/div/div[3]/div//p[4]')
    for i in grossv:
        try:
            gross.append(i.text.split("|")[1].split(":")[1])
        except:
            gross.append('NA')
    
    # scraping Url for each movie as popularity score is given inside the link.
    url1= driver.find_elements(By.XPATH, '//*[@id="main"]/div/div[3]/div//h3/a')
    for i in url1:
        urls.append(i.get_attribute("href"))
            
    try:
        if page==0:
            next_b=driver.find_element(By.XPATH, '/html/body/div[2]/div/div[2]/div[3]/div[1]/div/div[4]/a')
        else:
            next_b=driver.find_element(By.XPATH, '/html/body/div[2]/div/div[2]/div[3]/div[1]/div/div[4]/a[2]')
    except NoSuchElementException:
        break
    webdriver.ActionChains(driver).move_to_element(next_b).click(next_b).perform()    
    time.sleep(3)


In [35]:
# Scraping popularity score for each movie
for url in urls:
    driver.get(url)
    time.sleep(1)
    try:
        popular=driver.find_element(By.XPATH,'/html/body/div[2]/main/div/section[1]/section/div[3]/section/section/div[3]/div[2]/div[2]/div[1]/div/div[3]/a/span/div/div[2]/div[1]')        
        popularity.append(popular.text)
    except NoSuchElementException:
        popularity.append('NA')

In [37]:
len(popularity)

1756

In [75]:
# Creating first file
df1=pd.DataFrame({'Sno':sno,'Movie Name': m_name,'Directors':director,'Duration':runtime,'Year':year ,'Ratings':rating, 'Metascore':meta})

In [76]:
df1.head(5)

,Sno,Movie Name,Directors,Duration,Year,Ratings,Metascore
0,1,Spider-Man: Across the Spider-Verse,"Joaquim Dos Santos, Kemp Powers, Justin K. Th...",140 min,2023,9.0,86
1,2,The Dark Knight,Christopher Nolan,152 min,2008,9.0,84
2,3,The Lord of the Rings: The Return of the King,Peter Jackson,201 min,2003,9.0,94
3,4,Inception,Christopher Nolan,148 min,2010,8.8,74
4,5,The Lord of the Rings: The Fellowship of the Ring,Peter Jackson,178 min,2001,8.8,92


In [77]:
# Bifurcating the Director field into subfields as per the number of directors

df1[['director1', 'director2', 'director3','director4', 'director5', 'director6']] = df1['Directors'].str.split(',', expand=True)

In [78]:
df1.head()

,Sno,Movie Name,Directors,Duration,Year,Ratings,Metascore,director1,director2,director3,director4,director5,director6
0,1,Spider-Man: Across the Spider-Verse,"Joaquim Dos Santos, Kemp Powers, Justin K. Th...",140 min,2023,9.0,86,Joaquim Dos Santos,Kemp Powers,Justin K. Thompson,None,None,None
1,2,The Dark Knight,Christopher Nolan,152 min,2008,9.0,84,Christopher Nolan,None,None,None,None,None
2,3,The Lord of the Rings: The Return of the King,Peter Jackson,201 min,2003,9.0,94,Peter Jackson,None,None,None,None,None
3,4,Inception,Christopher Nolan,148 min,2010,8.8,74,Christopher Nolan,None,None,None,None,None
4,5,The Lord of the Rings: The Fellowship of the Ring,Peter Jackson,178 min,2001,8.8,92,Peter Jackson,None,None,None,None,None


In [81]:
# Drop main directors column since we have seperate column for directors 
data1= df1.drop('Directors', axis=1)

In [82]:
data1.head(5)

,Sno,Movie Name,Duration,Year,Ratings,Metascore,director1,director2,director3,director4,director5,director6
0,1,Spider-Man: Across the Spider-Verse,140 min,2023,9.0,86,Joaquim Dos Santos,Kemp Powers,Justin K. Thompson,None,None,None
1,2,The Dark Knight,152 min,2008,9.0,84,Christopher Nolan,None,None,None,None,None
2,3,The Lord of the Rings: The Return of the King,201 min,2003,9.0,94,Peter Jackson,None,None,None,None,None
3,4,Inception,148 min,2010,8.8,74,Christopher Nolan,None,None,None,None,None
4,5,The Lord of the Rings: The Fellowship of the Ring,178 min,2001,8.8,92,Peter Jackson,None,None,None,None,None


In [83]:
# Creating second file
df2=pd.DataFrame({'Movie Name': m_name,'Stars':star,'Votes':vote,'Genre':genre ,'Gross Collection':gross,'Popularity':popularity,'Certification': certificate})

In [84]:
df2.head(5)

,Movie Name,Stars,Votes,Genre,Gross Collection,Popularity,Certification
0,Spider-Man: Across the Spider-Verse,"Shameik Moore, Hailee Steinfeld, Brian Tyree ...","136,269","Animation, Action, Adventure",NA,2,PG
1,The Dark Knight,"Christian Bale, Heath Ledger, Aaron Eckhart, ...","2,727,109","Action, Crime, Drama",$534.86M,82,PG-13
2,The Lord of the Rings: The Return of the King,"Elijah Wood, Viggo Mortensen, Ian McKellen, O...","1,891,922","Action, Adventure, Drama",$377.85M,320,PG-13
3,Inception,"Leonardo DiCaprio, Joseph Gordon-Levitt, Elli...","2,420,290","Action, Adventure, Sci-Fi",$292.58M,71,PG-13
4,The Lord of the Rings: The Fellowship of the Ring,"Elijah Wood, Ian McKellen, Orlando Bloom, Sea...","1,920,460","Action, Adventure, Drama",$315.54M,153,PG-13


In [85]:
# Bifurcating the stars field into 4 subfields as per the number of stars worked


df2[['star1', 'star2', 'star3','star4'] ] = df2['Stars'].str.split(',', expand=True)

In [86]:
df2.head(5)

,Movie Name,Stars,Votes,Genre,Gross Collection,Popularity,Certification,star1,star2,star3,star4
0,Spider-Man: Across the Spider-Verse,"Shameik Moore, Hailee Steinfeld, Brian Tyree ...","136,269","Animation, Action, Adventure",NA,2,PG,Shameik Moore,Hailee Steinfeld,Brian Tyree Henry,Luna Lauren Velez
1,The Dark Knight,"Christian Bale, Heath Ledger, Aaron Eckhart, ...","2,727,109","Action, Crime, Drama",$534.86M,82,PG-13,Christian Bale,Heath Ledger,Aaron Eckhart,Michael Caine
2,The Lord of the Rings: The Return of the King,"Elijah Wood, Viggo Mortensen, Ian McKellen, O...","1,891,922","Action, Adventure, Drama",$377.85M,320,PG-13,Elijah Wood,Viggo Mortensen,Ian McKellen,Orlando Bloom
3,Inception,"Leonardo DiCaprio, Joseph Gordon-Levitt, Elli...","2,420,290","Action, Adventure, Sci-Fi",$292.58M,71,PG-13,Leonardo DiCaprio,Joseph Gordon-Levitt,Elliot Page,Ken Watanabe
4,The Lord of the Rings: The Fellowship of the Ring,"Elijah Wood, Ian McKellen, Orlando Bloom, Sea...","1,920,460","Action, Adventure, Drama",$315.54M,153,PG-13,Elijah Wood,Ian McKellen,Orlando Bloom,Sean Bean


In [87]:
# Bifurcating the genre into 3 subfields as per the number of genres

df2[['Genre1', 'Genre2', 'Genre3'] ] = df2['Genre'].str.split(',', expand=True)

In [88]:
df2.head(5)

,Movie Name,Stars,Votes,Genre,Gross Collection,Popularity,Certification,star1,star2,star3,star4,Genre1,Genre2,Genre3
0,Spider-Man: Across the Spider-Verse,"Shameik Moore, Hailee Steinfeld, Brian Tyree ...","136,269","Animation, Action, Adventure",NA,2,PG,Shameik Moore,Hailee Steinfeld,Brian Tyree Henry,Luna Lauren Velez,Animation,Action,Adventure
1,The Dark Knight,"Christian Bale, Heath Ledger, Aaron Eckhart, ...","2,727,109","Action, Crime, Drama",$534.86M,82,PG-13,Christian Bale,Heath Ledger,Aaron Eckhart,Michael Caine,Action,Crime,Drama
2,The Lord of the Rings: The Return of the King,"Elijah Wood, Viggo Mortensen, Ian McKellen, O...","1,891,922","Action, Adventure, Drama",$377.85M,320,PG-13,Elijah Wood,Viggo Mortensen,Ian McKellen,Orlando Bloom,Action,Adventure,Drama
3,Inception,"Leonardo DiCaprio, Joseph Gordon-Levitt, Elli...","2,420,290","Action, Adventure, Sci-Fi",$292.58M,71,PG-13,Leonardo DiCaprio,Joseph Gordon-Levitt,Elliot Page,Ken Watanabe,Action,Adventure,Sci-Fi
4,The Lord of the Rings: The Fellowship of the Ring,"Elijah Wood, Ian McKellen, Orlando Bloom, Sea...","1,920,460","Action, Adventure, Drama",$315.54M,153,PG-13,Elijah Wood,Ian McKellen,Orlando Bloom,Sean Bean,Action,Adventure,Drama


In [90]:
# Drop main stars, Genre columns since we have seperate column for stars, genres
data2= df2.drop(['Stars', 'Genre'], axis=1)

In [91]:
data2.head(5)

,Movie Name,Votes,Gross Collection,Popularity,Certification,star1,star2,star3,star4,Genre1,Genre2,Genre3
0,Spider-Man: Across the Spider-Verse,"136,269",NA,2,PG,Shameik Moore,Hailee Steinfeld,Brian Tyree Henry,Luna Lauren Velez,Animation,Action,Adventure
1,The Dark Knight,"2,727,109",$534.86M,82,PG-13,Christian Bale,Heath Ledger,Aaron Eckhart,Michael Caine,Action,Crime,Drama
2,The Lord of the Rings: The Return of the King,"1,891,922",$377.85M,320,PG-13,Elijah Wood,Viggo Mortensen,Ian McKellen,Orlando Bloom,Action,Adventure,Drama
3,Inception,"2,420,290",$292.58M,71,PG-13,Leonardo DiCaprio,Joseph Gordon-Levitt,Elliot Page,Ken Watanabe,Action,Adventure,Sci-Fi
4,The Lord of the Rings: The Fellowship of the Ring,"1,920,460",$315.54M,153,PG-13,Elijah Wood,Ian McKellen,Orlando Bloom,Sean Bean,Action,Adventure,Drama


In [92]:
# Converting to csv file
data1.to_csv('file1.csv')
data2.to_csv('file2.csv')